## Ejemplo de resolución del problema de Poisson con distintas condiciones de contorno ##


El problema a resolver es:


\begin{align}
-\Delta u &= f \;\;\;\;\; \text{in } \Omega \\
u &= g \;\;\;\;\; \text{in } \partial \Omega_{int} \\
\hat{n} \cdot \nabla u &= h \;\;\;\;\; \text{in } \partial \Omega_{ext} \\
\end{align}

Para ello impondremos la versión débil del mismo:

Encuentre $u$ en $H^1(\Omega, f))$ (o sea con las condiciones de contorno de Dirichlet en $\partial \Omega_{int}$) tal que,

$$
\int_{\Omega} \nabla v \cdot \nabla u \; d\Omega 
- \int_{\Omega} v \; f \; d\Omega 
- \oint_{\partial \Omega_{ext}} v \; h \; d\Gamma 
= 0 \;\;\;\;\; \forall v \;\; \in H^1_0(\Omega)
$$

Si obtenemos un $u$ satisfaciendo esta ecuación, y es suficientemente suave, entonces podemos integrar por partes el primer término y obtener:

$$
\int_{\Omega}  v \; (-\Delta u - f) \; d\Omega 
+ \oint_{\partial \Omega_{ext}} v \; (\hat{n} \cdot \nabla u - h) \; d\Gamma 
= 0 \;\;\;\;\; \forall v \;\; \in H^1_0(\Omega)
$$

Tomando $v$ arbitrario pero de soporte compacto vemos que $u$ debe satisfacer:

$$
-\Delta u = f \;\;\;\;\; \text{in } \Omega,
$$
y tomando $v$ arbitrario vemos que también se debe cumplir la condición de Neumann,

$$
\hat{n} \cdot \nabla u = h \;\;\;\;\; \text{in } \partial \Omega_{ext}.
$$

La condición de Dirichlet es automática por la elección del espacio.




Para resolver el problema utilizaremos la infraestructura del paquete `Gridap.jl` de Julia. Este ejemplo es una recopilación de varios ejemplos en el tutorial del paquete. 

In [2]:
import Pkg; Pkg.activate("gridap_makie")
using Gridap
using GridapGmsh
#mkdir("models")
#mkdir("images")

  Activating project at `~/github_repositories/my_repositories/pde2022/tareas/05_tarea/gridap_makie`


In [2]:
plot_s = false
if plot_s
    using GridapMakie, GLMakie #Para graficar 
    using FileIO #Gráficos y salidas
end

Vamos a usar una grilla previamente construida con la librería `gmsh`. En el directorio `models` encontrarán un *script* con terminación `.geo` (rectangle_hole.geo) que es el que se usó para construir el ejemplo. En base al mismo, y siguiendo el tutorial de `gmsh` podrán construir distintas grillas. También se pueden usar otras librerías para construir grillas. Estas se importan a la infraestructura **Gridap** y con ellas se construye la triangulación a usar. Notar que en el sript se dan nombres a las dos fronteras, la externa (rectangular), `"ext"` y le interna (círculo), `"int"`

In [3]:
#model = GmshDiscreteModel("models/rectangle_hole_fine.msh")
#model = GmshDiscreteModel("models/rectangle_hole_coarse.msh")
model = GmshDiscreteModel("models/rectangle_hole_finer.msh")


Info    : Reading 'models/rectangle_hole_finer.msh'...
Info    : 18 entities
Info    : 79392 nodes
Info    : 158784 elements
Info    : Done reading 'models/rectangle_hole_finer.msh'


UnstructuredDiscreteModel()

In [4]:
Ω = Triangulation(model)

BodyFittedTriangulation()

In [5]:
degree = 2
dΩ = Measure(Ω,degree)

Measure()

In [6]:
if plot_s
    fig, ax = plot(Ω)
    ax.aspect = AxisAspect(2)
    wireframe!(Ω, color=:black, linewidth=1)
    scatter!(Ω, marker=:star8, markersize=4, color=:blue)
    fig
end

### seleccionamos la condición de contorno a cumplir

In [6]:
full_dirichlet  = true
int_dirichlet   = false

false

Una vez que tenemos el grillado comenzamos a definir los elementos finitos que utilizaremos. En este caso usaremos elementos lagrangiano de **orden 1** que cumplirán una condición de Dirichlet en la región $\partial \Omega_{int}$. Al construirse la grilla esta región ha sido marcada como la frontera interior del rectángulo con el `tag` `"int"`. 

In [7]:
order = 2
reffe = ReferenceFE(lagrangian,Float64,order)
if int_dirichlet
    dirichlet_tags="int" 
elseif full_dirichlet
    dirichlet_tags=["int","ext"]
end
V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags = dirichlet_tags)

UnconstrainedFESpace()

En este ejemplo vamos a testear nuestro código utilizando una *solution*
que tomaremos como la función $u_e(x,y) =  4((x-x_0)^2 - (y-y_0)^3) - 5y$.

Así $\Delta u_e = -f = 8 - 24(y-y_0)$ y la condición de Neumann estará dada por $h = \hat{n} \cdot \nabla u_e$.

Sacamos ideas de solución exácta y fuente de la pagina oficial de Gridap [https://gridap.github.io/Tutorials/dev/pages/t002_validation/#Tutorial-2:-Code-validation-1](https://gridap.github.io/Tutorials/dev/pages/t002_validation/#Tutorial-2:-Code-validation-1)

In [23]:
function solution_and_source_01(param)
    x₀,y₀=param;
    ue(x) = 4*((x[1]-x₀)^2 - (x[2]-y₀)^3) - 5.0*x[2] # "exact solution"
    f(x) = -8.0 + 24*(x[2] - y₀)                     # source
    return ue,f
end

function solution_and_source_02()
    ue(x) = x[1]+x[2] # "exact solution"
    f(x) = 0.0   # source
    return ue,f
end

solution_and_source_02 (generic function with 1 method)

In [9]:
if plot_s
    fig, ax, plt = plot(Ω, ue, shading=false)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

In [24]:
ue,f=solution_and_source_02()
# internal Dirichlet boundary condition
U = TrialFESpace(V,ue)
Γ₁ = BoundaryTriangulation(model,tags=dirichlet_tags)
n₁ = get_normal_vector(Γ₁)

GenericCellField():
 num_cells: 1472
 DomainStyle: ReferenceDomain()
 Triangulation: BoundaryTriangulation()
 Triangulation id: 15565181124651048714

In [25]:
if int_dirichlet
    neumann_tags = "ext"
    #neumanntags = "int"
    Γ = BoundaryTriangulation(model,tags=neumann_tags)
    dΓ = Measure(Γ,degree)
    nb = get_normal_vector(Γ)
end

Para chequeo graficamos los valores de $\hat{n}\cdot \nabla u_e$ en el borde exterior.

In [12]:
if int_dirichlet && plot_s
    fig, ax , plt = plot(Γ, (nb ⋅ ∇(ue)), colormap=:algae, linewidth=10)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

A continuación definimos el problema débil en forma abstracta:

In [26]:
#h(x) =  #external Neumann bc.
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ  # en a(u,v) va toda la dependencia con u que es la incógnita. 
if full_dirichlet
    b(v) = ∫(v*f )*dΩ # aquí todo lo que es fuente. 
elseif int_dirichlet
    b(v) = ∫(v*f )*dΩ + ∫(v*(nb ⋅ ∇(ue)))*dΓ # aquí todo lo que es fuente. 
end

b (generic function with 1 method)

A partir de este punto el paquete **Gridap.jl** genera un sistema del tipo $Ax=b$ y lo resuelve para la versión elementos finitos de u.

In [27]:
op = AffineFEOperator(a,b,U,V)


AffineFEOperator()

In [28]:
ls = LUSolver()
solver = LinearFESolver(ls)

LinearFESolver()

In [29]:
uh = solve(solver,op)


SingleFieldFEFunction():
 num_cells: 157312
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 12931436991585569690

In [21]:
if plot_s
    fig, ax, plt = plot(Ω, uh, shading=false)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

In [30]:
if full_dirichlet
    writevtk(Ω,"images/solución_dir",cellfields=["uh_dir" => uh])
elseif int_dirichlet
    writevtk(Ω,"images/solución_newmann",cellfields=["uh_neu" => uh])
end

(["images/solución_dir.vtu"],)

Ahora vamos a validar la solución encontrada comparándola con la exacta, para ello introduciremos varias herramientas.

In [31]:
e = ue - uh

OperationCellField():
 num_cells: 157312
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 12931436991585569690

In [24]:
if plot_s
    fig, ax, plt = plot(Ω, e
        , shading=false
        )
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

In [32]:
if full_dirichlet
    writevtk(Ω,"images/error_dir",cellfields=["e_dir" => e])
elseif int_dirichlet
    writevtk(Ω,"images/error_newmann",cellfields=["e_neu" => e])
end

(["images/error_dir.vtu"],)

A continuación calculamos la norma $L^2$ y $H^1$ del error. 

In [33]:
el2 = sqrt(sum( ∫( e*e )*dΩ ))
println("l2 error = ",el2)
eh1 = sqrt(sum( ∫( e*e + ∇(e)⋅∇(e) )*dΩ ))
uh1 = sqrt(sum( ∫( uh*uh + ∇(uh)⋅∇(uh) )*dΩ ))
println("h1 error = ",eh1/uh1)

l2 error = 3.8029978946909316e-6
h1 error = 

0.001271578985145854


In [27]:
if plot_s
    fig, ax, plt = plot(Ω, ∇(e)⋅∇(e), shading=false)
    ax.aspect = AxisAspect(2)
    Colorbar(fig[1,2], plt)
    fig
end

# Mesh y Pseudocolor
![Una imagen con visIt](images/solucion_dir_mesh0000.jpeg)
![Una imagen con visIt](images/solucion_dir_pseudocolor0000.jpeg)

# Eror (pseudocolor)
![Una imagen con visIt](images/error_dir_pseudocolor0000.jpeg)